# SQLite向量数据库学习笔记

本笔记将介绍如何使用SQLite进行向量存储和KNN查询，并结合实际的embedding模型进行演示。我们将探索两种方法：
1. 使用sqlite-vec扩展（如果可用）
2. 使用纯Python实现的向量相似度计算（不依赖sqlite-vec）

## 1. 安装必要库

首先我们需要安装必要的库来支持向量操作和模型加载。

**注意**：sqlite-vec是一个C扩展，需要单独安装。可以从[GitHub releases](https://github.com/asg017/sqlite-vec/releases)下载预编译的二进制文件，或者按照[sqlite-vec文档](https://github.com/asg017/sqlite-vec)从源码编译。

In [ ]:
# 安装所需库
!pip install sentence-transformers numpy pandas scikit-learn

## 2. 导入必要的库

In [4]:
import sqlite3
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

c:\Users\k\.conda\envs\SmartImageFinder\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. 加载Embedding模型

我们将使用一个本地的embedding模型来生成向量。

In [5]:
try:
    # 首先尝试加载本地模型
    model_path = r'C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3'
    model = SentenceTransformer(model_path)
    print(f"成功加载本地模型: {model_path}")
except Exception as e:
    # 如果本地模型加载失败，使用Hugging Face上的小型模型
    print(f"本地模型加载失败: {e}")
    print("尝试从Hugging Face加载多语言模型...")
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    print("成功加载Hugging Face模型")

# 获取模型的输出维度
embedding_dim = model.get_sentence_embedding_dimension()
print(f"模型输出维度: {embedding_dim}")

成功加载本地模型: C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3
模型输出维度: 1024


## 4. 连接到SQLite数据库并尝试加载sqlite-vec扩展

首先我们将尝试加载sqlite-vec扩展。如果加载失败，我们将使用纯Python实现向量操作。

In [16]:
# 创建一个SQLite数据库连接
db_path = "vector_demo.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 尝试加载sqlite-vec扩展
vec_extension_available = False
try:
    conn.enable_load_extension(True)
    # 根据操作系统加载不同的扩展文件
    conn.execute("SELECT load_extension('./vec0.dll')")
    # 测试扩展是否成功加载
    cursor.execute("SELECT vec_version()")
    version = cursor.fetchone()[0]
    print(f"成功加载sqlite-vec扩展，版本: {version}")
    vec_extension_available = True
except Exception as e:
    print(f"加载sqlite-vec扩展失败: {e}")
    print("将使用纯Python实现向量操作")
    print("如需使用sqlite-vec扩展，请从 https://github.com/asg017/sqlite-vec/releases 下载对应版本")

成功加载sqlite-vec扩展，版本: v0.1.7-alpha.2


## 5. 准备向量数据存储

根据sqlite-vec扩展是否可用，我们将使用不同的方法来存储和查询向量数据。

我们将先创建一个辅助类来处理向量操作：

In [20]:
class VectorDatabase:
    def __init__(self, conn, vec_extension_available=False):
        self.conn = conn
        self.cursor = conn.cursor()
        self.vec_extension_available = vec_extension_available
        
        # 创建表
        self._create_tables()
    
    def _create_tables(self):
        """创建必要的表"""
        if self.vec_extension_available:
            # 如果扩展可用，尝试创建虚拟表
            try:
                self.cursor.execute(f"""
                    CREATE VIRTUAL TABLE IF NOT EXISTS vec_documents USING vec0(
                        document_id INTEGER PRIMARY KEY,
                        content_embedding FLOAT[{embedding_dim}] DISTANCE_METRIC=cosine,
                        category TEXT,
                        +original_content TEXT
                    );
                    """)
                print("成功创建vec0虚拟表")
            except Exception as e:
                print(f"创建虚拟表失败: {e}")
                print("将使用标准表")
                self.vec_extension_available = False
        
        # 创建标准表 (无论扩展是否可用)
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS documents(
            id INTEGER PRIMARY KEY,
            content TEXT,
            category TEXT,
            embedding BLOB
        );
        """)
        self.conn.commit()
    
    def insert_documents(self, documents):
        """插入文档"""
        # 清空表
        self.cursor.execute("DELETE FROM documents")
        if self.vec_extension_available:
            self.cursor.execute("DELETE FROM vec_documents")
        
        for doc in documents:
            # 标准表插入
            embedding_bytes = pickle.dumps(doc["embedding"])
            self.cursor.execute(
                "INSERT INTO documents(id, content, category, embedding) VALUES (?, ?, ?, ?)",
                (doc["id"], doc["content"], doc["category"], embedding_bytes)
            )
            
            # 虚拟表插入（如果可用）
            if self.vec_extension_available:
                embedding_json = json.dumps(doc["embedding"].tolist())
                try:
                    self.cursor.execute(
                        "INSERT INTO vec_documents(document_id, content_embedding, category, original_content) VALUES (?, vec_f32(?), ?, ?)",
                        (doc["id"], embedding_json, doc["category"], doc["content"])
                    )
                except Exception as e:
                    print(f"插入虚拟表失败: {e}")
        
        self.conn.commit()
        print(f"成功插入 {len(documents)} 条文档")
    
    def knn_search(self, query_embedding, k=3, category=None):
        """执行KNN搜索"""
        if self.vec_extension_available:
            # 使用vec0虚拟表
            try:
                query_json = json.dumps(query_embedding.tolist())
                if category:
                    self.cursor.execute("""
                    SELECT document_id, original_content, category, distance
                    FROM vec_documents 
                    WHERE content_embedding MATCH ? AND k = ? AND category = ?
                    """, (query_json, k, category))
                else:
                    self.cursor.execute("""
                    SELECT document_id, original_content, category, distance
                    FROM vec_documents 
                    WHERE content_embedding MATCH ? AND k = ?
                    """, (query_json, k))
                
                return [(row[0], row[1], row[2], row[3]) for row in self.cursor.fetchall()]
            except Exception as e:
                print(f"使用虚拟表搜索失败: {e}")
                print("将使用Python实现的向量搜索")
        
        # 使用Python实现的向量搜索
        # 获取所有文档
        if category:
            self.cursor.execute("SELECT id, content, category, embedding FROM documents WHERE category = ?", (category,))
        else:
            self.cursor.execute("SELECT id, content, category, embedding FROM documents")
        
        results = []
        for row in self.cursor.fetchall():
            doc_id, content, doc_category, embedding_bytes = row
            doc_embedding = pickle.loads(embedding_bytes)
            # 计算余弦相似度
            similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
            # 计算距离（1 - 相似度，使得值越小越相似）
            distance = 1 - similarity
            results.append((doc_id, content, doc_category, distance))
        
        # 排序并返回前k个结果
        results.sort(key=lambda x: x[3])  # 按距离排序
        return results[:k]

### 5.1 准备示例数据

首先，我们生成一些示例文本数据并计算它们的embedding向量。

In [21]:
# 准备一些示例文本
documents = [
    {"id": 1, "content": "机器学习是人工智能的一个子领域", "category": "技术"},
    {"id": 2, "content": "深度学习是机器学习的一种方法", "category": "技术"},
    {"id": 3, "content": "向量数据库可以高效存储和检索向量数据", "category": "数据库"},
    {"id": 4, "content": "SQLite是一个轻量级的关系型数据库", "category": "数据库"},
    {"id": 5, "content": "Python是一种流行的编程语言", "category": "编程"},
    {"id": 6, "content": "自然语言处理是处理人类语言的技术", "category": "技术"},
    {"id": 7, "content": "向量相似度搜索在推荐系统中很常用", "category": "技术"},
    {"id": 8, "content": "大数据分析需要高效的数据存储和处理", "category": "数据"}
]

# 为每个文档生成embedding向量
for doc in documents:
    embedding = model.encode(doc["content"])
    doc["embedding"] = embedding

# 展示部分数据
for doc in documents[:2]:
    print(f"ID: {doc['id']}, 内容: {doc['content']}")
    print(f"向量维度: {len(doc['embedding'])}, 向量前几个元素: {doc['embedding'][:5]}...\n")

ID: 1, 内容: 机器学习是人工智能的一个子领域
向量维度: 1024, 向量前几个元素: [-0.03649624 -0.02476022 -0.04679906 -0.00498728  0.00767139]...

ID: 2, 内容: 深度学习是机器学习的一种方法
向量维度: 1024, 向量前几个元素: [-0.02456879 -0.06428009 -0.04240257 -0.00734466 -0.02539387]...



### 5.2 初始化向量数据库并插入文档

In [22]:
# 初始化向量数据库
vector_db = VectorDatabase(conn, vec_extension_available)

# 插入文档
vector_db.insert_documents(documents)

成功创建vec0虚拟表
成功插入 8 条文档


## 6. 执行KNN查询

现在我们来演示如何执行K最近邻查询。

In [23]:
# 生成查询向量
query_text = "数据库技术与应用"
query_embedding = model.encode(query_text)

# 执行KNN查询
k = 3
results = vector_db.knn_search(query_embedding, k)

print(f"查询：'{query_text}'的最近{k}个结果:")
for doc_id, content, category, distance in results:
    print(f"ID: {doc_id}, 距离: {distance:.4f}, 类别: {category}, 内容: {content}")

查询：'数据库技术与应用'的最近3个结果:
ID: 3, 距离: 0.4323, 类别: 数据库, 内容: 向量数据库可以高效存储和检索向量数据
ID: 8, 距离: 0.4413, 类别: 数据, 内容: 大数据分析需要高效的数据存储和处理
ID: 4, 距离: 0.4911, 类别: 数据库, 内容: SQLite是一个轻量级的关系型数据库


### 6.1 使用类别过滤KNN查询

In [14]:
# 使用类别过滤KNN查询
category_filter = "技术"
results = vector_db.knn_search(query_embedding, k, category=category_filter)

print(f"查询：'{query_text}'的最近{k}个'{category_filter}'类别结果:")
for doc_id, content, category, distance in results:
    print(f"ID: {doc_id}, 距离: {distance:.4f}, 类别: {category}, 内容: {content}")

查询：'数据库技术与应用'的最近3个'技术'类别结果:
ID: 7, 距离: 0.5658, 类别: 技术, 内容: 向量相似度搜索在推荐系统中很常用
ID: 6, 距离: 0.5751, 类别: 技术, 内容: 自然语言处理是处理人类语言的技术
ID: 2, 距离: 0.5789, 类别: 技术, 内容: 深度学习是机器学习的一种方法


## 7. 手动实现KNN搜索（不使用扩展）

这部分演示如何使用纯Python实现KNN搜索，不依赖于sqlite-vec扩展。

In [14]:
def manual_knn_search(query_embedding, documents, k=3, distance_type='cosine'):
    """手动实现KNN搜索
    Args:
        query_embedding: 查询向量
        documents: 文档列表，每个文档包含embedding字段
        k: 返回的最近邻数量
        distance_type: 距离类型，'cosine'或'euclidean'
    Returns:
        前k个最相似的文档
    """
    results = []
    
    for doc in documents:
        if distance_type == 'cosine':
            # 计算余弦相似度
            similarity = cosine_similarity([query_embedding], [doc["embedding"]])[0][0]
            # 转换为距离（值越小越相似）
            distance = 1 - similarity
        else:  # 'euclidean'
            # 计算欧几里得距离
            distance = euclidean_distances([query_embedding], [doc["embedding"]])[0][0]
        
        results.append((doc["id"], doc["content"], doc["category"], distance))
    
    # 排序并返回前k个结果
    results.sort(key=lambda x: x[3])
    return results[:k]

In [15]:
# 使用余弦距离执行手动KNN搜索
results_cosine = manual_knn_search(query_embedding, documents, k=3, distance_type='cosine')

print(f"使用余弦距离手动查询：'{query_text}'的最近{k}个结果:")
for doc_id, content, category, distance in results_cosine:
    similarity = 1 - distance  # 将距离转换为相似度
    print(f"ID: {doc_id}, 相似度: {similarity:.4f}, 类别: {category}, 内容: {content}")

# 使用欧几里得距离执行手动KNN搜索
results_euclidean = manual_knn_search(query_embedding, documents, k=3, distance_type='euclidean')

print(f"\n使用欧几里得距离手动查询：'{query_text}'的最近{k}个结果:")
for doc_id, content, category, distance in results_euclidean:
    print(f"ID: {doc_id}, 距离: {distance:.4f}, 类别: {category}, 内容: {content}")

使用余弦距离手动查询：'数据库技术与应用'的最近3个结果:
ID: 3, 相似度: 0.5677, 类别: 数据库, 内容: 向量数据库可以高效存储和检索向量数据
ID: 8, 相似度: 0.5587, 类别: 数据, 内容: 大数据分析需要高效的数据存储和处理
ID: 4, 相似度: 0.5089, 类别: 数据库, 内容: SQLite是一个轻量级的关系型数据库

使用欧几里得距离手动查询：'数据库技术与应用'的最近3个结果:
ID: 3, 距离: 0.9298, 类别: 数据库, 内容: 向量数据库可以高效存储和检索向量数据
ID: 8, 距离: 0.9395, 类别: 数据, 内容: 大数据分析需要高效的数据存储和处理
ID: 4, 距离: 0.9910, 类别: 数据库, 内容: SQLite是一个轻量级的关系型数据库


## 8. SQLite-vec特性介绍

虽然我们可能无法在本环境中运行sqlite-vec的所有功能，但依然可以了解它提供的主要特性：

### 8.1 vec0虚拟表的列类型

sqlite-vec的虚拟表支持三种类型的列：
1. **元数据列**：普通列，可用于KNN查询的WHERE子句中进行过滤
2. **分区键列**：使用`PARTITION KEY`声明，用于分片向量索引，加速查询
3. **辅助列**：以`+`开头，存储额外数据，不能用于过滤但可直接检索

基本语法示例:
```sql
CREATE VIRTUAL TABLE vec_documents USING vec0(
    document_id INTEGER PRIMARY KEY,
    content_embedding FLOAT[1024] DISTANCE_METRIC=cosine,  -- 向量列
    category TEXT,                -- 元数据列
    user_id INTEGER PARTITION KEY, -- 分区键列
    +original_content TEXT        -- 辅助列
);
```

### 8.2 向量距离度量方式

sqlite-vec支持多种距离度量方式：
- `L2`（欧几里得距离）- 默认
- `L1`（曼哈顿距离）
- `cosine`（余弦距离）

### 8.3 向量格式

sqlite-vec支持多种向量格式：
- `vec_f32` - 32位浮点向量
- `vec_f64` - 64位浮点向量
- `vec_i32` - 32位整数向量
- `vec_i64` - 64位整数向量
- `vec_bit` - 二进制向量

## 9. 总结

通过本笔记，我们学习了：

1. 使用SQLite存储向量数据的两种方法：
   - 使用sqlite-vec扩展（如果可用）
   - 使用纯Python实现向量操作
   
2. 如何执行KNN查询以查找最相似的文档
   - 使用内置的余弦相似度和欧几里得距离
   - 根据类别等元数据过滤结果
   
3. SQLite-vec扩展的主要特性
   - vec0虚拟表的列类型
   - 支持的向量距离度量方式和向量格式

无论是否有sqlite-vec扩展，我们都可以使用SQLite作为一个简单的向量存储和检索系统，特别适合于小型应用和原型开发。

In [16]:
# 清理资源
conn.close()
print("数据库连接已关闭")

数据库连接已关闭
